In [67]:
import os
import json
import torch
from torch.utils.data import Dataset
from plyfile import PlyData
from glob import glob

class ScanNet200Dataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.scans = self._load_scans()

    def _load_scans(self):
        scans = []
        for scan_dir in os.listdir(self.root_dir): 
            scan_path = os.path.join(self.root_dir, scan_dir)
            if os.path.isdir(scan_path):
                ply_files = glob(os.path.join(scan_path, "*.ply"))
                json_files = glob(os.path.join(scan_path, "*.segs.json"))
                
                if ply_files and json_files:
                    ply_file = ply_files[0]
                    json_file = json_files[0]
                    scans.append((ply_file, json_file))
                else:
                    print(f"Missing files in {scan_path}")  # Debug print
        return scans

    def __len__(self):
        return len(self.scans)

    def _load_ply(self, ply_file):
        ply_data = PlyData.read(ply_file)
        vertices = torch.tensor(ply_data['vertex'].data.tolist(), dtype=torch.float32)
        return vertices

    def _load_json(self, json_file):
        with open(json_file, 'r') as f:
            data = json.load(f)
        segments = torch.tensor(data['segIndices'], dtype=torch.int64)
        return segments

    def __getitem__(self, idx):
        ply_file, json_file = self.scans[idx]
        vertices = self._load_ply(ply_file)
        segments = self._load_json(json_file)
        
        sample = {'vertices': vertices, 'segments': segments}
        
        if self.transform:
            sample = self.transform(sample)
        
        return sample

In [72]:
from path import Path

path = Path("../data/Scannet200/scans") 
dataset = ScanNet200Dataset(root_dir=path)
print(dataset.__len__())

1513


In [73]:
for data in dataset:
    print(data['vertices'].shape, data['segments'].shape)


torch.Size([122497, 7]) torch.Size([122497])
torch.Size([255083, 7]) torch.Size([255083])
torch.Size([97356, 7]) torch.Size([97356])


KeyboardInterrupt: 

In [74]:
from torch.utils.data import DataLoader

dataloader = DataLoader(dataset, batch_size=4, shuffle=True, num_workers=4)

for batch in dataloader:
    print(batch['vertices'].shape, batch['segments'].shape)


Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


Traceback (most recent call last):
  File "<string>", line 1, in <module>
  File "/Users/r.skorobogat/miniconda3/envs/adl4cv/lib/python3.12/multiprocessing/spawn.py", line 122, in spawn_main
    exitcode = _main(fd, parent_sentinel)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/r.skorobogat/miniconda3/envs/adl4cv/lib/python3.12/multiprocessing/spawn.py", line 132, in _main
    self = reduction.pickle.load(from_parent)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
AttributeError: Can't get attribute 'ScanNet200Dataset' on <module '__main__' (<class '_frozen_importlib.BuiltinImporter'>)>


KeyboardInterrupt: 